In [ ]:

import subprocess
import numpy as np
import matplotlib.pyplot as plt
import re
import boto3
import json
import os
import csv
import time

In [ ]:
import os
import openai
from openai import OpenAI
#from openai.error import RateLimitError

class OpenAIGPT:
    def __init__(self, model_id="gpt-4o-mini", api_key=os.getenv("CHATGPT_API_KEY"), delay=30, max_retries=20, max_time=360, count_tokens=False):
        self.model_id = model_id
        self.client = OpenAI(
            api_key=api_key
        )

    def execute(self, prompt, max_tokens=4096):
        chat_completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model=self.model_id,
        )

        return {
            "text": chat_completion.choices[0].message.content,
            "input_tokens": chat_completion.usage.prompt_tokens,
            "output_tokens": chat_completion.usage.completion_tokens
        }
llm = OpenAIGPT()

In [ ]:
tasks_generation_template = f""" 

You are an expert at analogue circuit design. You are required to decomposit tasks from initial user input. Here are some examples:

# Example 
'''
This is a circuit netlist, optimize this circuit with a gain above 20dB and bandwidth above 1Mag Hz.
'''

You can answer:
{{"questions": [
    {{"type_question": "Analyze this netlist and tell me the type of the circuit"}},
    {{"node_question": "This is a Spice netlist. Tell me the input and output node name. And the input voltage source name."}},
    {{"sim_question": "This is a Spice netlist for a circuit. Simulate it and give me the ac gain, bandwidth, and phase margin."}},
    {{"sizing_question": "Modify the parameter in the netlist. I want the gain above 20dB, bandwidth above 10000000Hz, and phase margin bigger than 60 degrees."}}

Please use the name above.
    
"""
tasks_generation_SYSTEM_PROMPT = tasks_generation_template

### Initial input

In [ ]:
tasks_generation_question = " This is a circuit netlist, optimize this circuit with a gain above 40dB, bandwidth above 1000000Hz, and phase margin bigger than 60 degree."
netlist = ''' 
.title Basic amp
M3 out1 out1 vdd vdd pmos1 W=1u L=0.09u
M4 out2 out1 vdd vdd pmos1 W=1u L=0.09u
M1 out1 in1 midp 0 nmos1 W=1u L=0.09u
M2 out2 in2 midp 0 nmos1 W=1u L=0.09u
M5 midp bias 0 0 nmos1 W=1u L=0.09u
M6 vdd out2 out 0 nmos1 W=1u L=0.09u
M7 out bias 0 0 nmos1 W=1u L=0.09u
Cc out2 out 1p
Cl out 0 10p
vbias bias 0 DC 0.3
vdd vdd 0 3.3
Vcm cm 0 DC 0.3
Eidp cm in1 diffin 0 1
Eidn cm in2 diffin 0 -1
Vid diffin 0 DC 0 AC 1 
.model nmos1 nmos level=14 version=4.8.1
.model pmos1 pmos level=14 version=4.8.1

.end

'''
tasks_generation_prompt = f""" 
system: {tasks_generation_SYSTEM_PROMPT},
human: Question: {tasks_generation_question}
Netlist: {netlist}
"""

In [ ]:
def get_tasks(tasks):
    tasks_output = json.loads(tasks["text"])
    for question_dict in tasks_output["questions"]:
        for key, value in question_dict.items():
            globals()[key] = value

In [ ]:
#tasks_generation_chain = tasks_generation_prompt | llm 
task_prompt = tasks_generation_prompt
print(task_prompt)
tasks = llm.execute(prompt = task_prompt)
#tasks = tasks_generation_chain.invoke(inputs)
print(tasks)

In [ ]:
get_tasks(tasks)

### Get target values

In [ ]:
target_value_question = tasks_generation_question
target_value_SYSTEM_PROMPT = ''' 
You are required to get the target value of some circuit performance from user input. 
Here is an example:
User input: 
    This is a circuit netlist, optimize this circuit with a gain above 30dB and bandidth above 10MHz.
Answer: 
{\n  "target_values": [\n    {\n      "gain_target": "20dB"\n    },\n    {\n      "bandwidth_target": "10000000Hz"\n    },\n    {\n      "phase_margin_target": "60degree"\n    }\n  ]\n}
Return a dictionary structured with a single key named "target_values" with no premable or explanation in JSON format, which maps to a list.  
This list contains multiple dictionaries, each representing a specific question. Each dictionary within the list has a key named by the 
name of the target performance and the content of the key is the value of the target performance in performance_target: value format.
'''
target_value_generation_prompt = f""" 
system: {target_value_SYSTEM_PROMPT},
human: Question: {target_value_question}
"""

target_value_prompt = target_value_generation_prompt
#print(task_prompt)
target_values = llm.execute(prompt = target_value_prompt)
#tasks = tasks_generation_chain.invoke(inputs)
print(target_values)

### Identify circuit type and generate corresponding prompt for simulation and optimization

In [ ]:
type_identify_template = ''' 
You are an expert at analogue circuit design. You are required to identify the circuit type from user netlist input. 
'''
type_SYSTEM_PROMPT = type_identify_template
type_identify_prompt = f""" 
system: {type_SYSTEM_PROMPT},
human: Question: {type_question}
Netlist: {netlist}
"""

In [ ]:
type_prompt = type_identify_prompt
type_identified = llm.execute(prompt = type_prompt)
print(type_identified)

### Input, output node and input source name extract

In [ ]:
def nodes_extract(node):
    node_name =  json.loads(node["text"])
    print(node_name)
    input_nodes = []
    output_nodes = []
    source_names = []

    for node in node_name['nodes']:
        if 'input_node' in node:
            input_nodes.append(node['input_node'])
        elif 'output_node' in node:
            output_nodes.append(node['output_node'])
        elif 'source_name' in node:
            source_names.append(node['source_name'])
    return input_nodes, output_nodes, source_names

def extract_code(text):
    # 正则表达式用于匹配三引号和反引号之间的代码
    regex1 = r"'''(.+?)'''" 
    regex2 = r"```(.+?)```"

    # 提取三引号和反引号之间的代码
    matches1 = re.findall(regex1, text, re.DOTALL)
    matches2 = re.findall(regex2, text, re.DOTALL)

    # 合并提取的代码
    extracted_code = "\n".join(matches1 + matches2)
    lines = extracted_code.split('\n')
    cleaned_lines = []

    # 清理提取的代码，移除含有 '*' 和 '#' 的部分
    for line in lines:
        if '*' in line:
            line = line.split('*')[0].strip()
        elif '#' in line:
            line = line.split('#')[0].strip()
        if line:  
            cleaned_lines.append(line)
    
    cleaned_code = "\n".join(cleaned_lines)
    return cleaned_code

node_SYSTEM_PROMPT = '''
You aim to find the circuit output nodes, input nodes and input voltage source name from the given netlist. 
Here is an example output:
{"nodes": [{"input_node": "in"},
                {"output_node": "out"},        
                {"source_name": "Vin"}]}
Please do no add any description in the output, just the dictionary.
'''
node_prompt = f""" 
system: {node_SYSTEM_PROMPT},
human: Question: {node_question}
Netlist: {netlist}
"""


In [ ]:
nodes = llm.execute(prompt = node_prompt)
print(nodes)

In [ ]:
input_nodes, output_nodes, source_names = nodes_extract(nodes)
print("----------------------node extract-----------------------------")
print(f"input_nodes:{input_nodes}")
print(f"output_nodes:{output_nodes}")
print(f"source_names:{source_names}")

### Select simulation type and call functions

In [ ]:
#fuctions for different simulation type
def dc_simulation(netlist, input_name, output_node):
     end_index = netlist.index('.end\n')

     #input_nodes_str = ' '.join(input_name)
     output_nodes_str = ' '.join(output_node)

     simulation_commands = f'''
            .dc {input_name} 0 1.8 0.001
            .control
              run         
              wrdata output_dc.dat {output_nodes_str}  
            .endc
     '''
     new_netlist = netlist[:end_index] + simulation_commands + netlist[end_index:]
     return new_netlist

def ac_simulation(netlist, input_name, output_node):
     end_index = netlist.index('.end\n')

     output_nodes_str = ' '.join(output_node)
     simulation_commands = f'''
      .ac dec 10 1 10G
      .control
        run         
        wrdata output_ac.dat {output_nodes_str} 
      .endc
     '''
     new_netlist = netlist[:end_index] + simulation_commands + netlist[end_index:]
     return new_netlist

def trans_simulation(netlist, input_name, output_node):
    end_index = netlist.index('.end\n')
    simulation_commands = f'''
            .trans 10ns 1us
            .control
              run         
              wrdata output_trans.dat {output_node} 
            .endc
     '''
    new_netlist = netlist[:end_index] + simulation_commands + netlist[end_index:]
    return new_netlist


In [ ]:
#fuctions for different results
Gain_init = None
Bw_init = None
Pm_init = None

def set_operating_point(file_name):
    data = np.genfromtxt(f'{file_name}.dat', skip_header=1)
    output = np.abs(data[:, 1]) #get output from np array
    input = data[:, 0] #get intput from np array
    index = np.argmin(np.abs(output - 0.9))
    corresponding_input = input[index]
    #print(corresponding_input)
    return corresponding_input

def ac_gain(file_name):
    data_ac = np.genfromtxt(f'{file_name}.dat', skip_header=1)
    num_columns = data_ac.shape[1]

    # for one output node
    if num_columns == 3:
        frequency = data_ac[:, 0]
        v_d = data_ac[:, 1] + 1j * data_ac[:, 2]
        gain = 20 * np.log10(np.abs(v_d[0]))
    # for 2 output nodes
    elif num_columns == 6:    
        v_d = data_ac[:, 4] + 1j * data_ac[:, 5]
        gain = 20 * np.log10(np.abs(v_d[0]))
    else:
        raise ValueError("The input file must have either 3 or 6 columns.")
    
    print(f"gain = {gain}")
    
    return gain

def bandwidth(file_name):
    data_ac = np.genfromtxt(f'{file_name}.dat', skip_header=1)
    num_columns = data_ac.shape[1]
    frequency = data_ac[:, 0]
    
    # for one output node
    if num_columns == 3:    
        v_d = data_ac[:, 1] + 1j * data_ac[:, 2]
        output = 20 * np.log10(v_d)
        gain = 20 * np.log10(np.abs(v_d[0]))
    # for 2 output nodes
    elif num_columns == 6:
        v_d = data_ac[:, 4] + 1j * data_ac[:, 5]
        output = 20 * np.log10(v_d)
        gain = 20 * np.log10(np.abs(v_d[0]))

    half_power_point = gain - 3
    
    indices = np.where(output >= half_power_point)[0]
    
    if len(indices) > 0:
        f_l = frequency[indices[0]]
        f_h = frequency[indices[-1]]
        bandwidth = f_h - f_l
    else:
        f_l = f_h = bandwidth = 0

    print(f"bandwidth = {bandwidth}")
    
    return bandwidth

def phase_margin(file_name):
    data_ac = np.genfromtxt(f'{file_name}.dat', skip_header=1)
    num_columns = data_ac.shape[1]
    frequency = data_ac[:,0]
    # for one output node
    if num_columns == 3:
        v_d = data_ac[:, 1] + 1j * data_ac[:, 2]   
    # for 2 output nodes
    elif num_columns == 6:    
        v_d = data_ac[:, 4] + 1j * data_ac[:, 5]
    #gain
    gain_db = 20 * np.log10(np.abs(v_d))
    #phase
    phase = np.degrees(np.angle(v_d))

    #find the frequency where gain = 0dB
    gain_db_at_0_dB = np.abs(gain_db - 0)
    index_at_0_dB = np.argmin(gain_db_at_0_dB)
    frequency_at_0_dB = frequency[index_at_0_dB]
    phase_at_0_dB = phase[index_at_0_dB]

    initial_phase = phase[0]
    tolerance = 5
    if np.isclose(initial_phase, 180, atol=tolerance):
        return phase_at_0_dB
    elif np.isclose(initial_phase, 0, atol=tolerance):
        return 180 - np.abs(phase_at_0_dB)
    else:
        raise ValueError("Unexpected initial phase value. Initial phase should be close to 0 or 180 degrees.")
        
def run_ngspice(circuit):
    with open('netlist.cir', 'w') as f:
        f.write(circuit)

    try:
        result = subprocess.run(['ngspice', '-b', 'netlist.cir'], capture_output=True, text=True)
        ngspice_output = result.stdout
    except Exception as e:
        ngspice_output = f"Error running NGspice: {str(e)}"

    print("NGspice output:")
    #print(ngspice_output)

def update_dc(circuit, new_dc_value):
    pattern = r"^\.param\s+vin\s*=[^\n]*\n?"
    
    # Remove matching lines
    replacement = f".param vin={new_dc_value}" + "\n"
    
    # Perform the substitution
    updated_netlist = re.sub(pattern, replacement, circuit, flags=re.MULTILINE)
    
    return updated_netlist

def tool_calling(tool_chain):
    global Gain_init, Bw_init, Pm_init
    gain = None
    bw = None
    pm = None
    sim_netlist = netlist
    
    for tool_call in tool_chain['tool_calls']:
        if tool_call['name'].lower() == "dc_simulation":
            sim_netlist = dc_simulation(sim_netlist, source_names, output_nodes)

        elif tool_call['name'].lower() == "ac_simulation":
            sim_netlist = ac_simulation(sim_netlist, source_names, output_nodes)
            print(f"ac_netlist:{sim_netlist}")

        elif tool_call['name'].lower() == "trans_simulation":
            sim_netlist = trans_simulation(sim_netlist, source_names, output_nodes)

        elif tool_call['name'].lower() == "set_operating_point":
            dc_value = set_operating_point('output_dc')

        elif tool_call['name'].lower() == "run_ngspice":
            run_ngspice(sim_netlist)

        elif tool_call['name'].lower() == "update_dc":
            sim_netlist = update_dc(netlist, dc_value)
            #print(f"tool call:{sim_netlist}")

        elif tool_call['name'].lower() == "ac_gain":   
            gain = ac_gain('output_ac')
            Gain_init = gain
            print(f"ac_gain result: {gain}")       
        elif tool_call['name'].lower() == "bandwidth":
            bw = bandwidth('output_ac')
            Bw_init = bw
            print(f"bandwidth result: {bw}")
        elif tool_call['name'].lower() == "phase_margin":
            pm = phase_margin('output_ac')
            Pm_init = pm
            print(f"phase margin: {pm}")


    sim_output = f"ac_gain is {gain}, " + f"bandwidth is {bw}, " + f"phase margin is {pm}"
# output the result as str for llm input
    return sim_output, sim_netlist

def extract_number(value):
        try:
            return float(''.join(filter(str.isdigit, value)))
        except ValueError:
            return None

In [ ]:
simulation_function_explanation = """
    
You have access to a set of functions you can use to answer the user's question. You should decide which tools to use from user input. 
and think about the question and return a chain of tools to use in sequence. 
When user ask to simulate or give specfic circuit performance, first think about which type of simulation should be done and which function in simulation functions can do this. 
Then, after select the simulation function, the netlist should be simulated by simulation tools run_ngspice.
Finaly, to analysis the output data file, a specfic function should be done to calculate and give the funal result. Choose the analysis functions to do this.
Here is an example:
{\n    "tool_calls": [{"name": "run_ngspice"},{"name": "ac_simulation"},}
Please do not add any comment in the output.
"""
simulation_function_defination = """
Here are the functions available for you to answer the question.

#Simulation functions:

dc_simulation: a function for expend dc simulation command to the given netlist. Return the modified netlist in string format between ''' '''.
ac_simulation: a function for expend ac simulation command to the given netlist. Return the modified netlist in string format between ''' '''.
trans_simulation: a function for expend transient simulation command to the given netlist. Return the modified netlist in string between ''' '''.

#Simulation tools:
run_ngspice: simulate the modified netlist and output the result in a file.

#Analysis tools:
ac_gain: calculate ac gain from .dat file and return the result.
bandwidth: calculate bandwidth from .dat file and return the result.
phase_margin: calculate phase margin from .dat file and return the result.

"""
simulation_SYSTEM_PROMPT = simulation_function_explanation + simulation_function_defination

In [ ]:

sim_prompt = f""" 
system:{simulation_SYSTEM_PROMPT}
human: Question: {sim_question}
Netlist: {netlist}
"""

In [ ]:
tools = llm.execute(prompt = sim_prompt)
print(tools)

In [ ]:
tool_chain = json.loads(tools["text"])
print("----------------------function used-----------------------------")
print(tool_chain)
sim_output, sim_netlist = tool_calling(tool_chain)
print("-------------------------result---------------------------------")
print(sim_output)
print("-------------------------netlist---------------------------------")
print(sim_netlist)

### Optimization loop

In [ ]:
analysing_template = f""" 
You are an expert at analogue circuit design. You are required to design CMOS amplifier. 
Please analysis their difference, and conclude on the relationship between specific parameters and performance and tell me the conclusion. e.g. Increase W of current mirror can increase the gain.

Let's think step by step to analysis the relationship:
Step1: detect the parameter difference in the provided netlist and output which were changed.
Step2: detect the performance difference.
Step3: Suggest possible relationship between different parameters and performance. You need to consider all the performance given to you. 
Please consider each subcircuit (differential pair, current mirror, current source, load...) and each parameters (W, L, C...) seperately because they are always with different size and have different impact on different performance.

You may output in this format:

Assistant: 
According to pervious results, I find that increase(decrease, or maintain) ... of differential pair (current mirror, current source, C)can increase(decrease, or maintain) gain.
increase(decrease, or maintain) ... of differential pair (current mirror, current source, C) can increase(decrease, or maintain)bandwidth.
increase(decrease, or maintain) ... of differential pair (current mirror, current source, C) can increase(decrease, or maintain)phase margin.
"""

#print(analysising_prompt)

In [ ]:
sizing_Question = f"Currently, {sim_output}. " + sizing_question
sizing_SYSTEM_PROMPT = f"""
You are an expert at analogue circuit design. You are required to design CMOS amplifier. Please follow the guide given to sizing component.
Please analysis the results given to you and tell me what you observed.
And according to your observation, you should deside which parameters should be changed to get the target result.
"""

sizing_output_template = f""" 
Notice: 
1. Do not change CL and input cm voltage. Always keep bias voltage within 0.6.
2. Please always change L with W to maintain a proper ratio within 100/1. The minimum L is 0.09u and max W is 500u.
3. Please return the complete netlist, and please do not change the simulation settings.
4. If one performance reach 90% of target, than please mainly consider the performance far away.
5. Please do not change simulation settings, source name, connections, node names. Please only change the values. Please do not use sub circuits.

Please change some parameters to see what happened every time and give the reasons why you change any parts of the circuit netlist. 
Finally, change these value in the netlist for further simulation, only one netlist is enough. 
Return the whole netlist with simulation settings and without comment between ''' ''', and give the reason why you change these parameters.

You may output in this format:
#Assistant: 
Current performance is ..., and according to my observation, ... is lower or higher than target. Let's make some adjustments to the circuit parameters:
1. gain: actions...
2. bandwidth: actions ...
3. phase margin: actions ...

#Here's the modified netlist with the proposed changes:
'''
'''
Please do not use other mark
Reasons for changes:
1.
2.
...  
"""

In [ ]:
optimising_template = f""" 
You are an expert at analogue circuit design. Please generate a detailed circuit optimisation guide based on observation provided to you. You should analysis the performance and the difference between component parameters. You should think about the relationship between parameters and performance. Please consider W, L, C, R seperately due to their different influence on different performance. You should output like in this format:

Assistant:
First stage: Gain is directly proportional to W of input transistors and L of load transistors...
Phase margin is proportional to ...
Bandwidth is ...
"""

In [ ]:
opti_function_defination = """
You have access to a set of functions you can use to answer the user's question. You should decide which tools to use from user input. 
and think about the question and return a chain of tools to use in sequence. 

Let's think step by step.
Step1: If the question given to you include the target performance and a paragraph with ''' ''' netlist inside, then call the extraxt code function.
Step2: When user ask to simulate or give specfic circuit performance, first think about which type of simulation should be done and which function in simulation functions can do this. 
Step3: after select the simulation function, the netlist should be simulated by simulation tools run_ngspice.
Step4: to analysis the output data file, a specfic function should be done to calculate and give the funal result. Choose the analysis functions to do this.

Here is an example of output format:
{\n    "tool_calls": [{"name": "run_ngspice"},{"name": "ac_simulation"},}
Please do not add any comment in the output.

Here are the functions available for you to answer the user input.

Extract_functions:
extract_code: extract complete netlist with simulation command from input text, always between ''' ''', and this netlist is for further simulation.

Simulation tools:
run_ngspice: simulate the modified netlist and output the result in a file.

Analysis tools:
ac_gain: calculate ac gain from .dat file and return the result.
bandwidth: calculate bandwidth from .dat file and return the result.
phase_margin: calculate phase margin from .dat file and return the result.

"""
opti_SYSTEM_PROMPT = simulation_function_explanation + opti_function_defination


In [ ]:
def optimization(target_values, sim_netlist):
    max_iterations = 20
    tolerance = 0.05  # 5% tolerance
    iteration = 0
    converged = False
    gain_output = None
    bw_output = None
    pm_output = None
    opti_output = None
    opti_netlist = sim_netlist
    previous_results = [f"{sim_output}, " + f",the netlist is {opti_netlist}"]
    gain_output_list = []
    bw_output_list = []
    pm_output_list = []
      
    target_output = json.loads(target_values["text"])
    for dict in target_output["target_values"]:
        for key, value in dict.items():
            globals()[key] = value
    
    gain_target = extract_number(globals().get('gain_target', '0')) if 'gain_target' in globals() else None
    bandwidth_target = extract_number(globals().get('bandwidth_target', '0')) if 'bandwidth_target' in globals() else None
    phase_margin_target = extract_number(globals().get('phase_margin_target', '0')) if 'phase_margin_target' in globals() else None
    
    gain_pass = True if gain_target not in globals() or gain_target is None else False
    bw_pass = True if bandwidth_target not in globals() or bandwidth_target is None else False
    pm_pass = True if phase_margin_target not in globals() or phase_margin_target is None else False

    sizing_Question = f"Currently, {sim_output}. " + sizing_question
    while iteration < max_iterations and not converged:
        time.sleep(60)
        print(f"----------------------iter = {iteration}-----------------------------")
        #print(f"previous_results:{previous_results}")
        ################################################
        analysising_prompt = f""" 
            system:{analysing_template}
            human: Previous:{previous_results}
        """
        analysis = llm.execute(prompt=analysising_prompt)
        print(analysis)
        analysis_content = analysis["text"]
        #################################################
        optimising_prompt = f""" 
            system:{optimising_template}
            Observation:{analysis_content}
        """
        optimising = llm.execute(prompt=optimising_prompt)
        print(optimising)
        optimised_prompt = optimising["text"]
        print(optimised_prompt)
        #################################################
        sizing_prompt = f""" 
            system:{sizing_SYSTEM_PROMPT}
            human: Question: {sizing_Question},
            Netlist: {opti_netlist}
            Please follow the instructions to update parameters in netlist:{optimised_prompt}, Output format: {sizing_output_template}
        """
        modified = llm.execute(prompt= sizing_prompt)
        print(modified)
        modified_output = modified["text"]
        print("----------------------Modified-----------------------------")
        print(modified_output)
        #################################################
        #opti_question = modified_output
        opti_prompt = f""" 
            system:{opti_SYSTEM_PROMPT}
            human: Question: {sizing_question + modified_output},
        """  
        modified_tools = llm.execute(prompt= opti_prompt)
        print(modified_tools)
        tools = json.loads(modified_tools["text"])
        print("----------------------tool use-----------------------------")
        print(tools)

        print("------------------------result-----------------------------")
        opti_netlist = extract_code(modified_output)
        print(opti_netlist)
        for tool_call in tools['tool_calls']:

            if tool_call['name'].lower() == "extract_code":
                opti_netlist = extract_code(modified_output)
                #print(opti_netlist)

            elif tool_call['name'].lower() == "run_ngspice":
                run_ngspice(opti_netlist)
                print("running ngspice")
        
            elif tool_call['name'].lower() == "ac_gain":
                #run_ngspice(sim_netlist)
                gain_output = ac_gain('output_ac')
                #print(f"ac_gain result: {gain_output}") 

            elif tool_call['name'].lower() == "bandwidth":
                #run_ngspice(sim_netlist)
                bw_output = bandwidth('output_ac')
                #print(f"bandwidth result: {bw_output}")

            elif tool_call['name'].lower() == "phase_margin":
                #run_ngspice(sim_netlist)
                pm_output = phase_margin('output_ac')
                #print(f"phase margin result: {pm_output}")
            
        opti_output = f"ac_gain is {gain_output}, " + f"bandwidth is {bw_output}, " + f"phase margin is {pm_output}"

        #save the output value in a list
        previous_results.append(f"Currently, {opti_output}, the netlist is {opti_netlist}")
        gain_output_list.append(gain_output)
        bw_output_list.append(bw_output)
        pm_output_list.append(pm_output)
                    
        print(opti_output)

        #comparison
        if gain_target is not None:
            if gain_output >= gain_target - gain_target * tolerance:
                gain_pass = True
            else:
                gain_pass = False
        
        if bandwidth_target is not None:
            if bw_output >= bandwidth_target - bandwidth_target * tolerance:
                bw_pass = True
            else:
                bw_pass = False

        if phase_margin_target is not None:
            if pm_output >= phase_margin_target - phase_margin_target * tolerance:
                pm_pass = True
            else:
                pm_pass = False

        if gain_pass and bw_pass and pm_pass:
            converged = True

        sizing_Question = f"Currently,{opti_output}, gain_pass:{gain_pass}, bandwidth_pass:{bw_pass}, phase_margin_pass:{pm_pass}" + sizing_question
        
        iteration += 1
        previous_results.append(f"Currently, {opti_output}, the netlist is {opti_netlist}")
        if len(previous_results) > 3:
            previous_results.pop(0)  # Remove the oldest result

        print(f"gain_target:{gain_target}, bandwidth_target:{bandwidth_target}, phase_margin_target:{phase_margin_target}")
        print(f"gain_pass:{gain_pass}, bandwidth_pass:{bw_pass}, phase_margin_pass:{pm_pass}")
    ##########################################################################################################
    # save the value in file
    file_empty = not os.path.exists('ota_results_history_mini.csv') or os.stat('ota_results_history_mini.csv').st_size == 0
    with open('ota_results_history_mini.csv', 'a', newline='') as csvfile:
        fieldnames = ['iteration', 'gain_output', 'bw_output', 'pm_output']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if file_empty:
            writer.writeheader()
        writer.writerow({'iteration': 0, 'gain_output': Gain_init, 'bw_output': Bw_init, 'pm_output': Pm_init})
        for i in range(len(gain_output_list)):
            writer.writerow({'iteration': i+1, 'gain_output': gain_output_list[i], 'bw_output': bw_output_list[i], 'pm_output': pm_output_list[i]})

    return {'converged': converged, 
            'iterations': iteration, 
            'gain_output': gain_output if 'gain_output' in locals() else None, 
            'bw_output': bw_output if 'bw_output' in locals() else None,
            'pm_output':pm_output if 'pm_output' in locals() else None}, opti_netlist
  

In [ ]:
def run_multiple_optimizations(target_values, sim_netlist, num_runs=1):
    results = []  # List to store results of each run
    
    for i in range(num_runs):
        print(f"Starting optimization run {i + 1}")
        result, opti_netlist = optimization(target_values, sim_netlist)
        results.append(result)  # Append the result of each run to the results list
        print(f"Run {i + 1} result: {result}")
        print("----------------------\n")
        time.sleep(30)
results = run_multiple_optimizations(target_values, sim_netlist)

# Print final summary of all runs
print("Summary of all optimization runs:")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
df = pd.read_csv('ota_results_history_4o.csv')
df['batch'] = (df['iteration'] == 0).cumsum()

plt.figure(figsize=(6, 5))
colors = plt.cm.viridis(np.linspace(0, 1, df['batch'].max() + 1))  # Generate a list of colors
for batch, group in df.groupby('batch'):
    plt.plot(group['iteration'], group['gain_output'], marker='o', color=colors[batch], label=f'Attempt {batch}')
plt.xlim(df['iteration'].min() - 1, df['iteration'].max() +1)
plt.ylim(df['gain_output'].min() - 1, df['gain_output'].max() +1)

# Fill the area beneath the line from minimum gain upwards
plt.fill_between(plt.xlim(), 38, plt.ylim()[1], color='blue', alpha=0.1, label='Target Range')

plt.title('OTA with history - Claude 3.5 Sonnet')
plt.xlabel('Iterations')
plt.ylabel('Gain(dB)')
plt.legend()
plt.grid(False)
plt.savefig('ota_gain.pdf', format='pdf')
plt.show()

plt.figure(figsize=(6, 5))
colors = plt.cm.viridis(np.linspace(0, 1, df['batch'].max() + 1))  # Generate a list of colors
for batch, group in df.groupby('batch'):
    plt.plot(group['iteration'], group['pm_output'], marker='o', color=colors[batch], label=f'Attempt {batch}')

plt.xlim(df['iteration'].min() - 1, df['iteration'].max() +1)
plt.ylim(df['pm_output'].min() - 1, df['pm_output'].max() +1)
plt.fill_between(plt.xlim(), 57, plt.ylim()[1], color='blue', alpha=0.1, label='Target Range')

plt.title('OTA with history - Claude 3.5 Sonnet')
plt.xlabel('Iterations')
plt.ylabel('Phase Margin(°)')
plt.legend()
plt.grid(False)
plt.savefig('ota_pm.pdf', format='pdf')
plt.show()

plt.figure(figsize=(6, 5))
colors = plt.cm.viridis(np.linspace(0, 1, df['batch'].max() + 1))  # Generate a list of colors
for batch, group in df.groupby('batch'):
    plt.plot(group['iteration'], group['bw_output'], marker='o', color=colors[batch], label=f'Attempt {batch}')
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x/1e6:.1f}')) #set the axis unit

plt.xlim(df['iteration'].min() - 1, df['iteration'].max() +1)
plt.ylim(df['bw_output'].min() - 1e6, df['bw_output'].max() +1e6)
plt.fill_between(plt.xlim(), 9.5e5, plt.ylim()[1], color='blue', alpha=0.1, label='Target Range')

plt.title('OTA with history - Claude 3.5 Sonnet')
plt.xlabel('Iterations')
plt.ylabel('Bandwidth(MHz)')
plt.legend()
plt.grid(False)
plt.savefig('ota_bw.pdf', format='pdf')
plt.show()